In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from scipy.spatial.distance import pdist
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.pyplot as plt

## Load Data

In [ ]:
# Read data from a tab-separated text file
data = pd.read_csv("../data/protein.txt", sep="\t")
data

## Prepare Data

In [ ]:
# Remove categorical columns and normalize (center and scale) the numerical variables
data_prep = data.drop(columns=['Country'])
scaler = StandardScaler()
data_prep_scaled = scaler.fit_transform(data_prep)

## K-Means Clustering

In [ ]:

# Perform k-means clustering with k=5
kmeans_cluster_model = KMeans(n_clusters=5, n_init=20, random_state=42)
kmeans_clusters = kmeans_cluster_model.fit_predict(data_prep_scaled)

# Add cluster assignments to the original data
data_prep_w_kmeans_clusters = data.copy()
data_prep_w_kmeans_clusters['kmeans_clusters'] = kmeans_clusters

# Get cluster centroids
centroids = kmeans_cluster_model.cluster_centers_.T

## Hierarchical Clustering

In [ ]:
# Perform hierarchical clustering
distances = pdist(data_prep_scaled, metric='euclidean')
h_cluster_model = linkage(distances, method='complete')

In [ ]:
plt.figure(figsize=(20, 7))
dendrogram(h_cluster_model, color_threshold=0, labels=data.iloc[:, 0].values) 
plt.title('Dendrogram for Hierarchical Clustering')
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.show()

In [ ]:
# Cut the dendrogram at k=5 and extract cluster assignments
k = 5
h_clusters = cut_tree(h_cluster_model, n_clusters=k).flatten()  # Flatten to get a 1D array
data_prep_w_h_clusters = data.copy()
data_prep_w_h_clusters['h_clusters'] = h_clusters

## Interpreting Clusters via their Centroids

In [ ]:
k = 7
h_clusters = cut_tree(h_cluster_model, n_clusters=k).flatten()  # Flatten to get a 1D array

# Create a DataFrame to hold the original data and cluster assignments
data_prep_w_h_clusters = data.copy()
data_prep_w_h_clusters['h_clusters'] = h_clusters

# Sort the DataFrame by the 'h_clusters' column
data_prep_w_h_clusters_sorted = data_prep_w_h_clusters.sort_values(by='h_clusters')
data_prep_w_h_clusters_sorted

In [ ]:


# Step 1: Calculate centroids
centroids = []
for cluster in range(k):
    cluster_points = data_prep_scaled[h_clusters == cluster]
    centroid = cluster_points.mean(axis=0)
    centroids.append(centroid)

centroids = np.array(centroids)

# Step 2: Create a DataFrame for centroids
centroids_df = pd.DataFrame(centroids, columns=data.columns[1:])  # Adjust if needed
centroids_df['Cluster'] = range(k)

# Step 3: Create bar plots for each cluster
for cluster in range(k):
    plt.figure(figsize=(10, 6))
    plt.bar(centroids_df.columns[:-1], centroids_df.iloc[cluster, :-1], color='skyblue')
    
    # Add labels and title
    plt.xlabel('Features')
    plt.ylabel('Distance from Centroid')
    plt.title(f'Distances from Centroid of Cluster {cluster}')
    plt.xticks(rotation=45)  # Rotate feature names for better visibility
    plt.grid(axis='y')
    
    # Show the plot
    plt.tight_layout()
    plt.show()

    # Step 4: Display the DataFrame for the corresponding cluster
    cluster_data = data_prep_w_h_clusters_sorted[data_prep_w_h_clusters_sorted['h_clusters'] == cluster]
    print(f'Data for Cluster {cluster}:')
    print(cluster_data)
    print('\n' + '-'*50 + '\n')  # Separator for clarity

## Plot Dendrogram After Cutting

In [ ]:

from scipy.cluster.hierarchy import fcluster

# Compute the flat clusters to confirm there are five clusters
num_clusters = 5
cluster_assignments = fcluster(h_cluster_model, t=num_clusters, criterion='maxclust')

# Find the appropriate distance threshold for two clusters
# The threshold is typically the height at which the tree splits into the desired number of clusters
color_threshold = h_cluster_model[-(num_clusters - 1), 2]  

# Plot the dendrogram with the color threshold
plt.figure(figsize=(20, 7))
dendrogram(
    h_cluster_model,
    labels=data['Country'].values,
    leaf_rotation=90,
    leaf_font_size=5,
    color_threshold=color_threshold
)
plt.title('Hierarchical Clustering Dendrogram (Colored by 5 Clusters)')
plt.axhline(y=color_threshold, c='black', linestyle='--', lw=1)  # Add a line for the cut-off
plt.show()

## Evaluate Clusterings

In [ ]:
k = 5
h_clusters = cut_tree(h_cluster_model, n_clusters=k).flatten()

# Calculate silhouette score
silhouette_avg = silhouette_score(data_prep_scaled, h_clusters)

# Calculate silhouette values for each sample
silhouette_values = silhouette_samples(data_prep_scaled, h_clusters)

# Print cluster sizes and average silhouette width for k=5
unique, counts = np.unique(h_clusters, return_counts=True)
cluster_sizes = dict(zip(unique, counts))

avg_silhouette_widths = {}
for cluster in unique:
    # Get the silhouette values for the current cluster
    cluster_silhouette_values = silhouette_values[h_clusters == cluster]
    
    # Calculate average silhouette width
    avg_silhouette_widths[cluster] = cluster_silhouette_values.mean()

print(f'Silhouette of {len(data)} units in {k} clusters:')
print('Cluster sizes and average silhouette widths:')
print(' '.join([str(cluster_sizes[cluster]) for cluster in unique]))
print(' '.join([f'{avg_silhouette_widths[cluster]:.6f}' for cluster in unique]))
print()

# Calculate and print statistics for individual silhouette widths
min_width = silhouette_values.min()
q1_width = np.percentile(silhouette_values, 25)
median_width = np.median(silhouette_values)
mean_width = silhouette_values.mean()
q3_width = np.percentile(silhouette_values, 75)
max_width = silhouette_values.max()

print('Individual silhouette widths:')
print(f'    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.')
print(f'{min_width:.6f} {q1_width:.6f} {median_width:.6f} {mean_width:.6f} {q3_width:.6f} {max_width:.6f}')

In [ ]:
for i in range(2, 21):
    h_clusters = cut_tree(h_cluster_model, n_clusters=i).flatten()
    
    # Calculate silhouette score
    silhouette_avg = silhouette_score(data_prep_scaled, h_clusters)
    
    print(f'k = {i}: Average Silhouette Score = {silhouette_avg:.4f}')